In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
cd /content/drive/MyDrive/Colab Notebooks/Github/Colab_Notebook

/content/drive/MyDrive/Colab Notebooks/Github/Colab_Notebook


In [18]:
!git config --global user.email 'inseop.hwang@hotmail.com'
!git config --global user.name 'Hot-Kimchi'

In [19]:
!git add .
!git status
!git add 8_3_사전\ 훈련된\ 모델\ 활용하기.ipynb

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   "8_3_\341\204\211\341\205\241\341\204\214\341\205\245\341\206\253 \341\204\222\341\205\256\341\206\253\341\204\205\341\205\247\341\206\253\341\204\203\341\205\254\341\206\253 \341\204\206\341\205\251\341\204\203\341\205\246\341\206\257 \341\204\222\341\205\252\341\206\257\341\204\213\341\205\255\341\206\274\341\204\222\341\205\241\341\204\200\341\205\265.ipynb"



In [20]:
!git commit -m 'without dat'
!git push -u origin main

[main a00297c] 데이터증식 사용하지 않는 빠른 특성 추출
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite "8_3_\341\204\211\341\205\241\341\204\214\341\205\245\341\206\253 \341\204\222\341\205\256\341\206\253\341\204\205\341\205\247\341\206\253\341\204\203\341\205\254\341\206\253 \341\204\206\341\205\251\341\204\203\341\205\246\341\206\257 \341\204\222\341\205\252\341\206\257\341\204\213\341\205\255\341\206\274\341\204\222\341\205\241\341\204\200\341\205\265.ipynb" (93%)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 5.73 KiB | 1.43 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/Hot-KimChi/Colab_Notebook.git
   298e6f5..a00297c  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.


>### 작은 이미지 데이터셋에 딥러닝을 적용하는 일반적이고 매우 효과적인 방법
- 사전 훈련된 모델 사용(pretrained model): ex) 1400만개의 레이블된 이미지 데이터 & 대규모 컨브넷
  - 1) 특성 추출
  - 2) 미세 조정

>#### 1) 사전 훈련된 모델을 사용한 특성 추출

In [7]:
from tensorflow import keras
from tensorflow.keras import layers

conv_base = keras.applications.vgg16.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(180, 180, 3)
)

58889256/58889256 [==============================] - 3s 0us/step


In [8]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 180, 180, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 180, 180, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 90, 90, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 90, 90, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 90, 90, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 45, 45, 128)       0     

>### Data download

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import gdown
gdown.download(id='18uC7WTuEXKJDDxbj-Jq6EjzpFrgE7IAd', output='dogs-vs-cats.zip')

Downloading...
From: https://drive.google.com/uc?id=18uC7WTuEXKJDDxbj-Jq6EjzpFrgE7IAd
To: /content/dogs-vs-cats.zip
100%|██████████| 852M/852M [00:22<00:00, 37.6MB/s]


'dogs-vs-cats.zip'

In [3]:
!unzip -qq dogs-vs-cats.zip
!unzip -qq train.zip

In [4]:
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

In [5]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


>### 최종 특성 맵의 크기 (5, 5, 512)
- 이 지점에서 2가지 방식이 가능
  - 1) 새로운 데이터셋에서 합성곱 기반 층을 실행, 출력을 넘파이 배열로 디스크 저장.
    - 저장된 데이터를 독립된 밀집 연결 분류기에 입력으로 사용
    - 합성곱 연산은 전체 과정 중, 가장 비싼 부분
    - 모든 입력 이미지에 합성곱 기반 층을 한번만 실행, 비용이 적게 소요되나 데이터 증식 사용 불가능
  - 2) 준비된 모델 위에 Dense 층을 쌓아 확장.
    - 입력 데이터에서 엔드-투-엔드로 전체 모델 실행
    - 모든 입력 이미지가 매번 합성곱 기반층을 통과되기에 데이터 증식 사용 가능
    - 이러한 방식때문에 비용이 휠씬 많이 발생.

>#### 1) 합성곱 기반 층을 실행 & 출력을 넘파이로 저장

In [11]:
## 데이터 증식을 사용하지 않는 빠른 특성추출
import numpy as np

def get_features_and_labels(dataset):
  all_features = []
  all_labels = []

  for images, labels in dataset:
    preprocessed_images = keras.applications.vgg16.preprocess_input(images)
    features = conv_base.predict(preprocessed_images)
    all_features.append(features)
    all_labels.append(labels)
  
  return np.concatenate(all_features), np.concatenate(all_labels)

train_features, train_labels = get_features_and_labels(train_dataset)
val_features, val_labels = get_features_and_labels(validation_dataset)
test_features, test_labels = get_features_and_labels(test_dataset)

1/1 [==============================] - 0s 24ms/step


In [10]:
train_features.shape

(2000, 5, 5, 512)

>#### 규제를 위해 드롭아웃 사용한 밀집 연결 분류기 정의 및 훈련

In [14]:
inputs = keras.Input(shape=(5, 5, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs, outputs)
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

callbacks = [keras.callbacks.ModelCheckpoint(
    filepath='feature_extraction.keras',
    save_best_only=True,
    monitor='val_loss')
]

history = model.fit(train_features, train_labels,
                    epochs=20, 
                    validation_data=(val_features, val_labels),
                    callbacks=callbacks)

Epoch 1/20
63/63 [==============================] - 2s 11ms/step - loss: 22.3398 - accuracy: 0.9260 - val_loss: 8.9057 - val_accuracy: 0.9520
Epoch 2/20
63/63 [==============================] - 0s 6ms/step - loss: 5.1527 - accuracy: 0.9690 - val_loss: 4.5195 - val_accuracy: 0.9760
Epoch 3/20
63/63 [==============================] - 0s 6ms/step - loss: 1.0260 - accuracy: 0.9915 - val_loss: 7.8235 - val_accuracy: 0.9650
Epoch 4/20
63/63 [==============================] - 0s 6ms/step - loss: 1.6389 - accuracy: 0.9850 - val_loss: 4.1561 - val_accuracy: 0.9750
Epoch 5/20
63/63 [==============================] - 0s 5ms/step - loss: 0.5594 - accuracy: 0.9945 - val_loss: 5.3641 - val_accuracy: 0.9750
Epoch 6/20
63/63 [==============================] - 0s 6ms/step - loss: 1.1342 - accuracy: 0.9915 - val_loss: 7.3053 - val_accuracy: 0.9650
Epoch 7/20
63/63 [==============================] - 0s 5ms/step - loss: 1.1086 - accuracy: 0.9925 - val_loss: 5.7374 - val_accuracy: 0.9760
Epoch 8/20
63/63 [